In [1]:
# DELIVERABLE FOR THE PROJECT:
# "KINEMATICS OF MOBILITY"
# David Pastor-Escuredo (LifeD Lab)
# Licencia MIT

In [2]:
#Copyright <2019> <David Pastor Escuredo>

#Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

#The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
import pandas as pd
import numpy as np
import json
from nltk.tokenize import word_tokenize
import re
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import collections
from sklearn.cluster import KMeans
from PIL import Image
import time
from datetime import datetime, timedelta, date
from os import listdir
from os.path import isfile, join
from geopandas import GeoDataFrame
from shapely.geometry import Point
import pickle
import math

%matplotlib inline

def getday(d):
    if d<10:
        ds='0'+str(d)
    else:
        ds=str(d)
    return ds

In [4]:
datapath='/home/davidpastor/TEF_mob/'
#datapath='./'
netpath=datapath+'nets/'
netdescpath=datapath+'nets_desc/'
trajpath=datapath+'trajs/'
trajdesc=datapath+'traj_net/'


desc=['d_out_degree','d_out_eigenvalue',
       'd_out_betweenness','d_dis_betweenness','d_cfbetweenness',
       'd_out_closeness','d_dis_closeness', 'd_cfcloseness', 
       'd_out_ave_flow',
       'd_out_std_flow','d_out_ave_distance']
ide=1
th=1
region='_bogota'
with open(trajdesc+'TVD'+'_'+desc[ide]+region+'_th'+str(th)+'.cnf', 'rb') as fpp:
        VD=pickle.load(fpp)

desc=['out_degree']
ide=0
        
with open(trajdesc+'TD'+'_'+desc[ide]+region+'.cnf', 'rb') as fpp:
        TD=pickle.load(fpp)
        


In [25]:
print(len(TD.keys()))
print(len(VD.keys()))
print(len(set(TD.keys())))
print(len(set(VD.keys())))

2760092
451847
2760092
451847


In [5]:
#Load the stats of each trajectory vector and make an histogram
desc=['d_ind', 'd_outd', 'd_in_degree', 'd_out_degree','d_in_eigenvalue','d_out_eigenvalue','d_in_betweenness','d_out_betweenness','d_dis_betweenness','d_cfbetweenness','d_in_closeness','d_out_closeness','d_dis_closeness', 'd_cfcloseness', 'distance', 'flow', 'd_time', 'ratio_time']
desc=['d_in_degree', 'd_out_degree','d_in_eigenvalue','d_out_eigenvalue',
       'd_in_betweenness','d_out_betweenness','d_dis_betweenness','d_cfbetweenness',
       'd_in_closeness','d_out_closeness','d_dis_closeness', 'd_cfcloseness', 
       'd_in_ave_flow','d_out_ave_flow',
       'd_in_std_flow','d_out_std_flow','d_in_ave_distance','d_out_ave_distance',
       'distance', 'flow', 'd_time']
desc=['d_out_degree','d_out_eigenvalue',
       'd_out_betweenness','d_dis_betweenness','d_cfbetweenness',
       'd_out_closeness','d_dis_closeness', 'd_cfcloseness', 
       'd_out_ave_flow',
       'd_out_std_flow','d_out_ave_distance']
ic=0
col=['darkkhaki','firebrick', 'navy', 'olive', 'purple']
stat=['mean','max', 'min', 'median', 'std']
datapath='/home/davidpastor/TEF_mob/'
datapath='./'
netpath=datapath+'nets/'
netdescpath=datapath+'nets_desc/'
trajpath=datapath+'trajs/'
trajdesc=datapath+'traj_net/'

region='_bogota'
th=1

UD={}

for ide in range(0,len(desc)):
      
    with open(trajdesc+'TVD'+'_'+desc[ide]+region+'_th'+str(th)+'.cnf', 'rb') as fpp:
        VD=pickle.load(fpp)
     
    for u in VD:
        if u not in UD:
            UD[u]={}
            UD[u][desc[ide]]=[]
        UD[u][desc[ide]]=VD[u][stat[ic]]
        
dfUD = pd.DataFrame(UD)
print(dfUD.head())
    

                     8956009378    8915454644    8972103925    8955613665  \
d_out_degree       1.334403e-19 -2.343622e-05  6.074668e-03 -3.942553e-20   
d_out_eigenvalue  -2.497661e-22 -3.648095e-08  6.368919e-07 -3.368881e-23   
d_out_betweenness  0.000000e+00 -1.844902e-05  1.523259e-03 -7.885107e-20   
d_dis_betweenness  2.846726e-17 -3.783489e-04  3.628523e-02  1.340468e-18   
d_cfbetweenness    5.560011e-21  8.084632e-06  7.115167e-04  9.856383e-21   

                     8965483753    8967446095    8915404558    8973087716  \
d_out_degree       2.109260e-04  1.917509e-04  3.157574e-05 -2.409284e-05   
d_out_eigenvalue  -8.300518e-10  5.421155e-08  1.921038e-08 -3.530824e-09   
d_out_betweenness  1.081218e-03  5.201040e-05  9.650720e-06 -8.573758e-08   
d_dis_betweenness  3.666676e-04 -9.852387e-05 -3.599149e-07  2.416678e-05   
d_cfbetweenness    4.088661e-04 -2.740921e-05  3.382468e-06 -2.824840e-06   

                     8957909218    8911216251  ...  8911826987  8976050201

In [12]:
#OJO NO DEBERIA HABER NAN
print(len(VD.keys()))
print(len(desc))
print(dfUD.shape)

dfUD=dfUD.dropna(axis=1)
print(dfUD.shape)

451847
11
(11, 451847)
(11, 451846)


In [13]:
M=dfUD.to_numpy()
print(np.shape(M))

(11, 451846)


In [18]:
nc=3
Mt=M.transpose()
km = KMeans(n_clusters=nc, random_state=0).fit(Mt)
kl=km.labels_
print(len(kl))

451846


In [19]:
userlabel={}
users=dfUD.columns
c=0
for u in users:
    userlabel[u]=kl[c]
    c=c+1
    
with open(trajdesc+'UVD_label_'+str(nc)+region+'_th'+str(th)+'.cnf', 'wb') as handle:
    pickle.dump(userlabel, handle, protocol=pickle.HIGHEST_PROTOCOL) 

In [23]:
#Project onto map the statistics of a decriptor at the trajectory level

ide=0
desc=['cfbetweenness']
#desc=['ind', 'outd', 'in_degree', 'out_degree','in_eigenvalue','out_eigenvalue','in_betweenness','out_betweenness','dis_betweenness','cfbetweenness','in_closeness','out_closeness','dis_closeness', 'cfcloseness']
ic=0
col=['darkkhaki','firebrick', 'navy', 'olive', 'purple']
stat=['mean','max', 'min', 'median', 'std']
datapath='/home/davidpastor/TEF_mob/'
#datapath='./'
netpath=datapath+'nets/'
netdescpath=datapath+'nets_desc/'
trajpath=datapath+'trajs/'
trajdesc=datapath+'traj_net/'

with open(trajdesc+'TD'+'_'+desc[ide]+region+'.cnf', 'rb') as fpp:
    VD=pickle.load(fpp) 

with open(trajpath+'HL.tff', 'rb') as fpp:
    HL=pickle.load(fpp) 
    
ant_file='antennas/antennas.csv'
sheet=pd.read_csv(ant_file,delimiter=';')
print(sheet.head())
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
sheet['antenna_id']=LAC+Cell
    
vector=[]
vectoru=[]
vectorh=[]
for u in VD:
    vector.append(VD[u][stat[ic]])
    vectoru.append(u)
    vectorh.append(HL[u]['homeloc'])

print(len(vector))
print(len(vectoru))
    
df = pd.DataFrame(list(zip(vectoru, vector, vectorh)), columns =['user', desc[ide], 'home']) 

locations=df['home'].values
Hvalues={}

print(len(locations))
mv_v=[]
stdv_v=[]
lon=[]
lat=[]
for h in locations:
    df_h=df[df['home']==h]
    v=df_h[desc[ide]].values
    m_v=np.nanmean(v)
    std_v=np.nanstd(v)
    mv_v.append(m_v)
    stdv_v.append(std_v)
    reg=sheet[sheet['antenna_id']==h]
    lon.append(reg['LONGITUD'].values[0])
    lat.append(reg['LATITUD'].values[0])
    
df2 = pd.DataFrame(list(zip(average, std, location, lon, lat)), columns =['average', 'std', 'antenna', 'l1', 'l2']) 
print(df2.head())
print(len(df2))
df2.to_csv('TDstats_'+desc[ide]+'_'+stat[ic]+region+'.csv',index=False)

   LATITUD  LONGITUD Celda_HEX LAC_HEX
0   4.6092  -74.0718      2711    083F
1   4.6092  -74.0718      2712    083F
2   4.6092  -74.0718      2713    083F
3   4.6092  -74.0718      2714    083F
4   4.6092  -74.0718      2715    083F
2760092
2760092
2760092


/usr/lib/python3/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/numpy/lib/nanfunctions.py:1667: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


KeyboardInterrupt: 

In [ ]:
#Project onto map

ide=0
desc=['cfbetweenness']
#desc=['ind', 'outd', 'in_degree', 'out_degree','in_eigenvalue','out_eigenvalue','in_betweenness','out_betweenness','dis_betweenness','cfbetweenness','in_closeness','out_closeness','dis_closeness', 'cfcloseness']
ic=0
col=['darkkhaki','firebrick', 'navy', 'olive', 'purple']
stat=['mean','max', 'min', 'median', 'std']
datapath='/home/davidpastor/TEF_mob/'
netpath=datapath+'nets/'
netdescpath=datapath+'nets_desc/'
trajpath=datapath+'trajs/'
trajdesc=datapath+'traj_net/'

limx1=0.0005
limx2=0.002

with open(trajdesc+'TD'+'_'+desc[ide]+region+'.cnf', 'rb') as fpp:
    VD=pickle.load(fpp) 

with open(trajpath+'HL.tff', 'rb') as fpp:
    HL=pickle.load(fpp) 
    
vector=[]
vectoru=[]
for u in VD:
    vector.append(VD[u][stat[ic]])
    vectoru.append(u)

print(len(vector))
print(len(vectoru))
    
df = pd.DataFrame(list(zip(vectoru, vector)), columns =['user', desc[ide]]) 

su1=df[df[desc[ide]]>=limx1]
su2=su1[su1[desc[ide]]<=limx2]

users=su2['user'].values
userhome=[]
lon=[]
lat=[]

ant_file='antennas/antennas.csv'
sheet=pd.read_csv(ant_file,delimiter=';')
print(sheet.head())
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
sheet['antenna_id']=LAC+Cell

print(len(users))
for u2 in users:
    a=HL[u2]['homeloc']
    userhome.append(a)
    reg=sheet[sheet['antenna_id']==a]
    lon.append(reg['LONGITUD'].values[0])
    lat.append(reg['LATITUD'].values[0])
    
df2 = pd.DataFrame(list(zip(users, userhome, lon, lat)), columns =['user', 'antenna', 'l1', 'l2']) 
print(df2.head())
print(len(df2))
df2.to_csv('TD_'+desc[ide]+'4.csv',index=False)

ucount=[]
lon2=[]
lat2=[]
alist=set(userhome)
print(len(list(alist)))

for a1 in alist:
    au=df2[df2['antenna']==a1]
    count=len(au)
    ucount.append(count)
    reg2=sheet[sheet['antenna_id']==a1]
    lon2.append(reg2['LONGITUD'].values[0])
    lat2.append(reg2['LATITUD'].values[0])

df3 = pd.DataFrame(list(zip(ucount, alist, lon2, lat2)), columns =['nusers', 'antenna', 'l1', 'l2']) 
print(df3.head())
df3.to_csv('TD_'+desc[ide]+'_usercount4.csv',index=False)
print(len(df3))
    

In [5]:
#Project onto map

ide=0
region='_medellin'
limx1=0.001
limx2=0.002

desc=['d_in_degree']
#desc=['ind', 'outd', 'in_degree', 'out_degree','in_eigenvalue','out_eigenvalue','in_betweenness','out_betweenness','dis_betweenness','cfbetweenness','in_closeness','out_closeness','dis_closeness', 'cfcloseness']
ic=0
col=['darkkhaki','firebrick', 'navy', 'olive', 'purple']
stat=['mean','max', 'min', 'median', 'std']
datapath='/home/davidpastor/TEF_mob/'
netpath=datapath+'nets/'
netdescpath=datapath+'nets_desc/'
trajpath=datapath+'trajs/'
trajdesc=datapath+'traj_net/'



with open(trajdesc+'TVD'+'_'+desc[ide]+region+'_th1.cnf', 'rb') as fpp:
    VD=pickle.load(fpp) 

with open(trajpath+'HL.tff', 'rb') as fpp:
    HL=pickle.load(fpp) 
    
vector=[]
vectoru=[]
for u in VD:
    vector.append(VD[u][stat[ic]])
    vectoru.append(u)

print(len(vector))
print(len(vectoru))
    
df = pd.DataFrame(list(zip(vectoru, vector)), columns =['user', desc[ide]]) 

su1=df[df[desc[ide]]>=limx1]
su2=su1[su1[desc[ide]]<=limx2]

users=su2['user'].values
userhome=[]
lon=[]
lat=[]

ant_file='antennas/antennas.csv'
sheet=pd.read_csv(ant_file,delimiter=';')
print(sheet.head())
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
sheet['antenna_id']=LAC+Cell

print(len(users))
for u2 in users:
    a=HL[u2]['homeloc']
    userhome.append(a)
    reg=sheet[sheet['antenna_id']==a]
    lon.append(reg['LONGITUD'].values[0])
    lat.append(reg['LATITUD'].values[0])
    
df2 = pd.DataFrame(list(zip(users, userhome, lon, lat)), columns =['user', 'antenna', 'l1', 'l2']) 
print(df2.head())
print(len(df2))
df2.to_csv('TVDcensus_'+desc[ide]+region+'_th1.csv',index=False)

ucount=[]
lon2=[]
lat2=[]
alist=set(userhome)
print(len(list(alist)))

for a1 in alist:
    au=df2[df2['antenna']==a1]
    count=len(au)
    ucount.append(count)
    reg2=sheet[sheet['antenna_id']==a1]
    lon2.append(reg2['LONGITUD'].values[0])
    lat2.append(reg2['LATITUD'].values[0])

df3 = pd.DataFrame(list(zip(ucount, alist, lon2, lat2)), columns =['nusers', 'antenna', 'l1', 'l2']) 
print(df3.head())
df3.to_csv('TVDcount_'+desc[ide]+region+'_th1.csv',index=False)
print(len(df3))

88944
88944
   LATITUD  LONGITUD Celda_HEX LAC_HEX
0   4.6092  -74.0718      2711    083F
1   4.6092  -74.0718      2712    083F
2   4.6092  -74.0718      2713    083F
3   4.6092  -74.0718      2714    083F
4   4.6092  -74.0718      2715    083F
4617
         user   antenna       l1       l2
0  8958547353  070C4A62 -72.6422  2.57120
1  8951109542  13D0C649 -75.5940  6.26608
2  8918028825  13D1C44D -75.5836  6.24694
3  8918327555  13D1CB22 -75.5560  6.23583
4  8978623782  13C6C961 -75.5475  6.29333
4617
1402
   nusers   antenna       l1       l2
0       1  100EA4A9 -76.2302  3.99089
1       3  13D1D049 -75.5840  6.22813
2       1  13C6C53F -75.5489  6.27939
3       1  13CECB58 -75.5943  6.24550
4       6  13972DCA -75.5681  6.24842
1402
